# Cost of Plutocracy

This article aims to analyze the top 60 DAOs (ranked by [treasury size](https://deepdao.io/)) to determine the impact of large token holders on their governance.

In [1]:
# sets up the pynb environment
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

Our data sources include DeepDAO and Snapshot, both of which specialize in providing data on Decentralised Autonomous Organization (DAO) governance. We also use Coingecko to find market data for DAO governance tokens.

Using this data, we compile two spreadsheets that act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last fifty proposals in each DAO. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters that have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "../plutocracy_report.xlsx", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "../plutocracy_report_filtered.xlsx", sheet_name=None, engine="openpyxl"
)

We then ask the following question: **How many whales have voted in the last fifty proposals for each of the DAOs we analyzed?** Voting power for each of these DAOs varies by the type of [voting strategy](https://docs.snapshot.org/strategies/what-is-a-strategy) outlined in their respective Snapshot spaces.

However, to keep this analysis simple, we assume the common `erc-20-balance` strategy and define the cost of voting power (`vp`) as the US dollar value of a DAO's native token at the time of the proposal.

In [3]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
        "avg vp for non-whales",
        "avg vp for whales",
        "avg cost of vote",
    ],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
Uniswap,808,21605,1.626079,8.365904e+04,7.184673
ENS,321,9742,281.564683,5.561220e+04,29.874161
Lido,164,4438,115.166136,1.363135e+06,1.573386
Frax,6,173,81348.700379,1.481606e+07,9.052249
Curve Finance,14,235,3454.929793,1.352748e+06,3.338636
Decentraland,61,1307,202.882560,2.738316e+05,0.712545
Radicle,18,395,26779.601995,9.430134e+05,12.567908
Gitcoin,89,3223,17.509963,2.666339e+05,3.037398
Hop,30,1454,869.936796,4.161286e+05,0.101721
Gearbox,32,881,86802.863340,1.176725e+07,0.018553


For the DAOs above, the top 5% of voters have a disproportionate amount of voting power compared to the average voter.

We also have an insight into the economic might whales hold in their respective DAO. We calculate the average cost to purchase a token to vote at the time the proposal was active (in USD) for comparative purposes.

---

Next, we take a look at how governance decisions are affected once whale votes are discounted. We do so by comparing the scores of each proposal and checking whether the outcome of the proposal is changed when whales are filtered out.

We check whether a proposal's outcome changes by checking if the largest vote choice score changes after filtering out whales. Specifically, in python we do the following:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

In [5]:
pd.set_option("display.max_rows", 5)
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_differences_dfs = dict()

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "outcome_old",
                "outcome_new"
            ],
        )
        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization].index = score_differences_dfs[organization].index.to_series().apply(
            lambda s: f'<a href=http://snapshot.org/#/{space_id}/proposal/{s} rel="noopener noreferrer" target="_blank">{s[0:9]}</a>'
        )


## Uniswap

### Filtered Proposals Analysis

In [7]:
HTML(score_differences_dfs["Uniswap"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion","total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
QmP3H8WGL,0.999959,1.636272e+07,False,Add 1bps fee tier (1 tick),Add 1bps fee tier (1 tick)
0xee48516,0.999941,3.523730e+07,False,Yes,Yes
Qmcs5pAv7,0.999887,6.422912e+06,True,No,Yes
0x7f196c3,0.999886,1.370368e+07,False,Yes,Yes
0xab1dbb6,0.999859,1.533712e+07,False,Yes,Yes
0xe9f8e5d,0.999859,1.866476e+07,False,"Yes, implement pilot as outlined","Yes, implement pilot as outlined"
0x4fdc5d5,0.999850,1.222219e+07,False,Yes,Yes
0xe869bc6,0.999848,4.453477e+07,False,Yes,Yes
0xd70b3e8,0.999842,1.619916e+07,False,Yes,Yes


### Proportion of Outcomes Changed

Using [pandas](https://pandas.pydata.org/), we see that **12.5% of proposal outcomes change after removing whales.**



In [8]:
score_differences_dfs["Uniswap"]["outcome_changed"].value_counts(normalize=True)

False    0.875
True     0.125
Name: outcome_changed, dtype: float64

### Proposal Analysis

The outcome of the proposal to add a Liquidity Mining Manager (LMM) for the [Optimism-Uniswap Liquidity Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c) changes after removing whales.

If not for whale intervention, "Overnight Finance" would have been chosen as the LMM. Instead, "DeFiEdge" was selected. From the data, we see that without whale intervention, DeFiEdge would've had ~2,708,025 fewer votes out of the total ~2,728,492 voting power on this proposal. That is ~99.3% of voting power allocated for this proposal directed to choosing "DeFiEdge".

In [9]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Uniswap"].index.to_series().apply(
    lambda s: "0xfd3d380" in s,
)
proposal_score_differences = score_differences_dfs["Uniswap"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differences,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. LayerZero", the runner-up, received ~37.7% of the total whale voting power. By comparison, "Wormhole" received ~51.1%.

In [10]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Uniswap"].index.to_series().apply(
    lambda s: "0x6b8df36" in s,
)
proposal_score_differences = score_differences_dfs["Uniswap"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,2.840391e+07,1.717334e+07,662.606204,3772.802474
Score Differences,2.327546e+07,1.716363e+07,481.545728,3402.436704


## ENS

### Filtered Proposal Outcome Analysis

In [11]:
HTML(score_differences_dfs["ENS"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x41b3509,0.998301,1.670185e+06,False,For,For
0xd7eff78,0.998220,1.670154e+06,False,For,For
0x5788bf0,0.998020,1.671687e+06,False,For,For
0xa245dc7,0.998011,2.282886e+06,False,slobo.eth,slobo.eth
0xa714c25,0.997780,2.299997e+06,False,avsa.eth,avsa.eth
0xdaff050,0.997736,2.325021e+06,False,nick.eth,nick.eth
0x5c96e49,0.994136,2.126140e+06,False,For,For
0x46c7294,0.993740,2.137811e+06,False,For,For
0x4a1aedb,0.993403,2.639657e+06,False,For,For


### Proportion of Outcomes Changed

In [12]:
score_differences_dfs["ENS"]["outcome_changed"].value_counts(normalize=True)

False    0.964286
True     0.035714
Name: outcome_changed, dtype: float64

### Proposal Analysis

A very small percentage of proposal outcomes were changed after filtering out whales, around 3.5%. However, if we take look at score differences as they compare to each score, we see that almost all proposals were heavily backed by whales.

In [13]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_choices']
mask = score_differences_dfs["ENS"].index.to_series().apply(
    lambda s: "0x41b3509" in s,
)
proposal_score_differences = score_differences_dfs["ENS"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,For,Against,Abstain
Scores,1.669865e+06,125.702762,194.817417
Score Differences,1.667211e+06,0.000000,137.124541


[This proposal](https://snapshot.org/#/ens.eth/proposal/0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7) approves the funding of various grants ENS had committed to funding—-**250k USDC and 50 ETH of commitments in total**. 99.9% of the voting power committed to this proposal voted to approve it, 0.0001% was allocated to abstaining, and an even smaller percentage voted against it.

Meaning that ~0.1% of the vote would've still been allocated towards voting for the proposal to pass even without whale intervention, which I think many would see as fine since the funding of public goods is always well received by DAO communities.

[This proposal](https://snapshot.org/#/ens.eth/proposal/0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b) to commit **935k USDC and 254 ETH** to the ENS Ecosystem Working Group also enjoyed significant backing from whales. The Working Group is responsible for *growing and improving the ENS Ecosystem by funding people and projects that are ENS-specific or ENS-centric*.

In [14]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_choices']
mask = score_differences_dfs["ENS"].index.to_series().apply(
    lambda s: "0x5788bf0" in s,
)
proposal_score_differences = score_differences_dfs["ENS"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,For,Against,Abstain
Scores,1.449464e+06,208582.378002,13640.353424
Score Differences,1.446509e+06,208348.531592,13519.449465


Similar to the previous proposal, this was highly influenced by whales (~86.5% of voting power committed to this proposal committed by whales that voted for it to pass, ~12.5% from whales that voted against it, and ~0.8% of whales voted to abstain).

Yet, even after removing whales, the proposal still passes. After doing some quick math, we can see the remaining voting power for this proposal that voted for its passing equates to about ~0.17% out of the remaining 0.2% of voting power after removing whales.

Still, I do not believe the outcome of this proposal is controversial. An organisation funding initiatives that aid in the development of its product should not be a controversial topic for its members. 

## Lido
### Filtered Proposal Analysis

In [15]:
HTML(score_differences_dfs["Lido"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x3b1e5f9,0.999968,5.518937e+07,False,For,For
0x1db8052,0.999959,5.542913e+07,False,For,For
0x1454174,0.999956,6.425261e+07,False,For,For
0x7ac2431,0.999945,5.769020e+07,False,For,For
0xf4beaba,0.999944,6.042543e+07,False,For,For
0x8bbd48f,0.999936,3.686686e+07,False,For,For
0x32f6f09,0.999934,5.717509e+07,False,For,For
0x3436b98,0.999932,5.717666e+07,False,For,For
0x7f19fed,0.999930,5.455752e+07,False,YAY,YAY


### Proportion of Outcomes Changed

In [16]:
score_differences_dfs["Lido"]["outcome_changed"].value_counts(normalize=True)

False    0.96
True     0.04
Name: outcome_changed, dtype: float64

Here, there was a change in outcome for just two of the proposals (4%). [This proposal](https://snapshot.org/#/lido-snapshot.eth/proposal/0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b) to [expand the oracle set and quorum](https://mainnet.lido.fi/#/lido-dao/0x442af784a788a5bd6f42a01ebe9f287a871243fb/) of Lido DAO oracle node operators was passed on 16/12/22, onboarded 4 new oracle node operators, and set the quorum to 5/9 identical oracle reports to be accepted by the protocol.

Had whales not intervened, Option 1 to onboard 6 additional oracles and set the quorum to 6/11, would've passed.

In [17]:
propsal_choices = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Lido"].index.to_series().apply(
    lambda s: "0xcbf5343" in s,
)
proposal_score_differences = score_differences_dfs["Lido"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,"1. For 6 new oracles, 6/11 set","2. For 4 new oracles, 5/9 set",3. Against
Scores,96886.616711,5.272470e+07,4778.954352
Score Differences,92934.922644,5.272312e+07,4535.772055


Here we see whale participation was consistent for all choices for this proposal. In fact, ~99.982% of voting power allocated to this proposal came from whales that voted for each of the outcomes. The second outcome alone had ~99.980% of the voting power allocated to this proposal and so removing it would make a huge impression on the outcome.

Voting power allocated to this proposal would be low without whale intervention: (only ~9277.7397 LDO would be allocated to this proposal). However, the community would still vote for adding more oracles that [broadcasts the chain state from the beacon chain to the execution layer](https://docs.lido.fi/guides/oracle-operator-manual) of ETH 2.0.

## Frax
### Filtered Proposal Outcome Analysis

In [18]:
HTML(score_differences_dfs["Frax"].astype({"total_vp": "float64"}, copy=False).sort_values("whale_vp_proportion", ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x9547dee,0.999550,2.659437e+07,False,For,For
0xc44bf39,0.997379,2.715716e+07,False,For,For
0x5d2e624,0.997305,2.715915e+07,False,For,For
0x4540ed6,0.996720,2.717503e+07,False,For,For
0x1127ca8,0.985974,3.375063e+07,False,For,For
0x1c844e1,0.984243,3.377246e+07,False,For,For
0x8f497b3,0.983152,2.775706e+07,False,For,For
0xd7271b0,0.980319,3.099628e+07,False,For,For
0x52d8c99,0.980156,3.097254e+07,False,For,For


### Proportion of Outcomes Changed

In [19]:
score_differences_dfs["Frax"]["outcome_changed"].value_counts(normalize=True)

False    0.96
True     0.04
Name: outcome_changed, dtype: float64

While enormous voting power was filtered out from all the proposals listed, just one had their outcomes changed (representing 2% of all proposals analysed).

If we look at the [proposal which attracted the most voting power](https://snapshot.org/#/frax.eth/proposal/0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a), which renewed the 2023 grant for Flywheel (in the amount of approx $214.2k, split 50/50 in FXS and FRAX) for the promotion of "the Frax ecosystem and producing high-quality DeFi content", we see that just over 50% of the voting power was removed from this proposal after filtering out whales (approx 56.7%).

In [20]:
propsal_choices = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Frax"].index.to_series().apply(
    lambda s: "0xece8d5b" in s,
)
proposal_score_differences = score_differences_dfs["Frax"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,For,Against
Scores,1.172897e+07,131.88977
Score Differences,9.238258e+06,0.00000


Though, because the opposition voting power for this proposal was a paltry 1E-5% of the vote, filtering out whales did not change the outcome. After observing the number of voting participants (FXS token holders who participated in this proposal), this was not a controversial outcome.

## Decentraland
Let's now look at Decentraland, where the largest holders regularly participate in governance and dominate voting.

### Filtered Proposals Analysis

In [21]:
HTML(score_differences_dfs["Decentraland"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x7acae7a,0.999533,4.372915e+06,True,no,yes
0x8b84875,0.999486,4.883961e+06,True,no you can forget about it matey,yes I want someone to pick this up down the line
0x7a6a819,0.999382,5.481279e+06,False,yes,yes
0x187559b,0.999296,5.161839e+06,False,yes,yes
0x8668746,0.999199,5.172165e+06,True,yes,no
0x4913bba,0.998996,7.775241e+06,False,This is a good idea,This is a good idea
0x3262d5a,0.998879,4.874952e+06,False,no,no
0xdfc9d87,0.998616,1.130732e+06,True,no,yes
0x768ee01,0.998182,5.183137e+06,True,yes,no


### Proportion of Outcomes Changed

In [22]:
score_differences_dfs["Decentraland"]["outcome_changed"].value_counts(normalize=True)

False    0.74
True     0.26
Name: outcome_changed, dtype: float64

Here we see that just under a quarter of proposals' outcomes would change if whales were not involved in Decentraland governance.

For example, [this proposal](https://snapshot.org/#/snapshot.dcl.eth/proposal/0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5) to set a duration period for the tenure of Decentraland DAO committee members saw 99.9% of voting power attributed to whales, with 94.85% of proposal voting power allocated to voting for the proposal not to pass.

In [23]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Decentraland"].index.to_series().apply(
    lambda s: "0x7f6fed8" in s,
)
proposal_score_differences = score_differences_dfs["Decentraland"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Yes,No,Invalid question/options
Scores,168338.243827,1.394610e+06,0.025206
Score Differences,166209.638630,1.392896e+06,0.000000


This is clearly a case of large holders voting to support their own interests. Once whales are filtered out of the votes (which would've given existing and future committee members set terms, making the roles more democratic) the proposal would've passed, albeit, with very low voting power by comparison.

## Curve Finance

### Filtered Proposals Analysis

In [24]:
HTML(score_differences_dfs["Curve Finance"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xa11d4d1,0.998352,2.609730e+07,False,Yes,Yes
0xa4fb747,0.997837,7.183114e+03,False,当然,当然
0xad241d0,0.995149,1.352219e+05,True,>80%,<10%
0xdaae7dd,0.993704,2.424709e+03,False,Oui,Oui
0xd946d40,0.992450,2.056291e+03,False,会,会
0xf18e081,0.992417,2.027366e+03,True,不会,会
0x745856d,0.992091,2.131107e+04,False,"Sure, I am diamond hand","Sure, I am diamond hand"
0x8752a03,0.991968,2.161588e+03,False,YES,YES
0xaec626e,0.991251,1.727484e+03,True,NO,YES


### Proportion of Outcomes Changed

In [25]:
score_differences_dfs["Curve Finance"]["outcome_changed"].value_counts(normalize=True)

False    0.75
True     0.25
Name: outcome_changed, dtype: float64

One proposal which would've passed if not for whale intervention was this proposal to [add the XSTUSD-3CRV pair](https://snapshot.org/#/curve.eth/proposal/0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5) to Curve's [gauge](https://resources.curve.fi/reward-gauges/understanding-gauges) [controller](https://curve.readthedocs.io/dao-gauges.html#the-gauge-controller) to accrue CRV for liquidity providers of XSTUSD-3CRV.

XSTUSD is a stablecoin deployed on Polkadot and Kusama that's backed by a synthetic token called XOR (Sora's native token). The [governance discussion](https://gov.curve.fi/t/proposal-to-add-xstusd-3crv-to-the-gauge-controller/2998/15) about the vote was particularly interesting.

![](./res/curve_governance_shenanigans.png)

What stands out is XSTUSD's comparison with LUNA/UST. This proposal was created before the LUNA/UST [depegging disaster](https://rekt.news/luna-rekt/), but even before that, [quite a](https://twitter.com/runekek/status/1478166276979793922) [few people](https://twitter.com/FreddieRaynolds/status/1463960623402913797) had their concerns about it. So I checked out the first 16 accounts which showed really strong support for this proposal, and almost [every single](https://gov.curve.fi/u/meowtopia) [one was](https://gov.curve.fi/u/LiquidityKing) [created within](https://gov.curve.fi/u/Ryandotrrr) 2 days of the proposal's forum post. Clear signs of governance forum shenanigans, executed to raise hype for a proposal.

In [26]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Curve Finance"].index.to_series().apply(
    lambda s: "0x0eb23ea" in s,
)
proposal_score_differences = score_differences_dfs["Curve Finance"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Yes,No
Scores,45352.592228,529201.282019
Score Differences,23462.994316,511330.472493


Just over 4% of voting power for this proposal was allocated by whales to vote "Yes" (which is just over half the total voting power allocated to the "Yes" choice for this proposal), whereas ~89% of whale voting power was allocated to voting "NO" (~97% of total voting power for this choice).

It's important to highlight that whales are also sensible and not always "evil", which I would classify as entities that promote proposals that are detrimental to the DAO for their own interests. It should also be noted that this was, by far, the proposal with the most voting power allocated. It's a good thing that CRV whales didn't have an incentive to pass this proposal as well.

## Radicle
### Filtered Proposals Analysis

In [27]:
HTML(score_differences_dfs["Radicle"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
QmVypVPUP,0.927434,1.962803e+06,False,For,For
QmbpshrWc,0.751147,1.291031e+06,False,Unpause swapping,Unpause swapping
0xe9158c0,0.632033,7.565920e+05,False,Yes,Yes
0x704025e,0.624782,4.946306e+06,False,For,For
0x16400b2,0.621619,4.971479e+06,False,For,For
0x646e927,0.605303,6.163640e+06,False,Yes,Yes
QmepPgXwo,0.551155,2.012627e+05,True,Nay,Aye
QmW64iqHQ,0.539397,8.865300e+05,False,For,For
0xf95b0e0,0.483829,4.133690e+06,False,For,For


We see that 18% of proposal (2 proposals) outcomes change once whale votes are removed.

### Proportion of Outcomes Changed

In [28]:
score_differences_dfs["Radicle"]["outcome_changed"].value_counts(normalize=True)

False    0.818182
True     0.181818
Name: outcome_changed, dtype: float64

One such proposal was the one to [distribute RAD remaining](https://snapshot.org/#/gov.radicle.eth/proposal/QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S) following a Liquidity Bootsrapping (LBP) round conducted in February '21. This leftover RAD was proposed to be redistributed to participants of the LBP, i.e. people who bought RAD in this period from the Balancer LBP for RAD tokens.

In [29]:
propsal_choices = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Radicle"].index.to_series().apply(
    lambda s: "QmepPgXwo" in s,
)
proposal_score_differences = score_differences_dfs["Radicle"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Aye,Nay
Scores,78136.278442,123126.401817
Score Differences,20708.270722,90218.650933


Just over 10% of whale voting power for this proposal was allocated to voting "Aye" on this proposal (~27% of voting power allocated to the "Aye" choice came from whales). Whereas ~45% of voting power from whales was allocated to the "Nay" choice (~73% of whale voting power allocated to "Nay").

## Gitcoin
### Filtered Proposals Analysis

In [30]:
HTML(score_differences_dfs["Gitcoin"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x44711d0,0.999678,9.749282e+06,False,YES,YES
0x8a9aecd,0.999631,9.143112e+06,False,YES,YES
0xc6655f1,0.999620,1.196768e+07,False,Yes - Proceed with TD,Yes - Proceed with TD
0x643408b,0.999597,9.300537e+06,False,YES,YES
0xc32ebb4,0.999539,8.099869e+06,False,YES,YES
0x43aa826,0.999534,1.121580e+07,False,YES,YES
0x63e139b,0.999432,9.095121e+06,False,Yes,Yes
0x08bc5d6,0.999431,1.270002e+07,False,Yes,Yes
0x35010ae,0.999387,1.054562e+07,False,"Yes, fund MMM Season 16","Yes, fund MMM Season 16"


### Proportion of Outcomes Changed

In [32]:
score_differences_dfs["Gitcoin"]["outcome_changed"].value_counts(normalize=True)

False    0.9
True     0.1
Name: outcome_changed, dtype: float64

## Hop
### Filtered Proposals Analysis

In [33]:
HTML(score_differences_dfs["Hop"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xdf8dbd4,0.990016,1.319066e+07,False,For,For
0x8131ad6,0.988627,1.214940e+07,False,Yes,Yes
0x6b51364,0.986493,1.355622e+07,False,Yes,Yes
0x77dfa74,0.986203,1.356527e+07,False,Yes,Yes
0xf1bf932,0.986168,1.356350e+07,False,Yes,Yes
0x6873b06,0.986155,1.097978e+07,False,For,For
0x29c0205,0.984798,1.198519e+07,False,Yes,Yes
0x15937d8,0.984153,1.056193e+07,False,Yes,Yes
bafkreigy,0.982840,1.230943e+07,True,No,Yes


### Proportion of Outcomes Changed

In [34]:
score_differences_dfs["Hop"]["outcome_changed"].value_counts(normalize=True)

False    0.954545
True     0.045455
Name: outcome_changed, dtype: float64

## Gearbox
### Filtered Proposals Analysis

In [35]:
HTML(score_differences_dfs["Gearbox"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xa24c991,0.911661,2.266455e+08,False,"Yes, approve","Yes, approve"
0xc7d4eb1,0.898677,2.119328e+08,False,"Yes, approve","Yes, approve"
0x7155b39,0.891847,2.120120e+08,False,"Yes, approve","Yes, approve"
0x3cbd502,0.888188,2.444278e+08,False,"Yes, approve","Yes, approve"
0x2d77246,0.884413,2.471702e+08,False,Approve,Approve
0x5e348d3,0.876182,2.545753e+08,False,"Yes, approve","Yes, approve"
0x3a4f32b,0.876123,2.444973e+08,False,"Yes, approve","Yes, approve"
0x3719083,0.872143,2.331051e+08,False,Approve OBRA,Approve OBRA
0x24c3f73,0.825484,2.210429e+08,False,"Yes, update the Uniswap adapters","Yes, update the Uniswap adapters"


### Proportion of Outcomes Changed

In [37]:
score_differences_dfs["Gearbox"]["outcome_changed"].value_counts(normalize=True)

False    1.0
Name: outcome_changed, dtype: float64